# Testing the Utility Functions

In [1]:
import numpy as np


## Geting parameters for a certain observation time

### get_p_at_t()

In [15]:
from few.trajectory.inspiral import EMRIInspiral
from few.trajectory.ode import KerrEccEqFlux
from few.trajectory.ode import SchwarzEccFlux
from few.utils.utility import get_p_at_t

traj_module = EMRIInspiral(func=KerrEccEqFlux)

In [18]:
M = 1e6
mu = 1e2
a = 0.9  # This model supports a spinning primary compact object
p0 = 10.0
e0 = 0.8
xI0 = 1.0  # +1 for prograde, -1 for retrograde inspirals

T = 1.0  # duration of trajectory in years (as defined by few.utils.constants.YRSID_SI)

traj_pars = [M, mu, a, p0, e0, xI0]

t, p, e, xI, Phi_phi, Phi_theta, Phi_r = traj_module(*traj_pars, T=T)
p[-1]


np.float64(2.4314901991706392)

In [ ]:
# Looks like there's a more basic problem here.
# Test a single trajectory

In [ ]:


# set initial parameters
#M = 9700484.894655073
#mu = 25.806654874357793
#e0 = 0.7735415084736439
#a= 0.6834440767965327
from few.trajectory.ode import KerrEccEqFlux
from few.trajectory.ode import SchwarzEccFlux

traj_module = EMRIInspiral(func=KerrEccEqFlux)

M = 1e6
mu = 5e1
e0 = 0.5
a = 0.5
x = 1.

traj_args = [M, mu, a, e0, x]
traj_kwargs = {}
#index_of_p = 3

t_out = 1.
# run trajectory
p_new = get_p_at_t(
    traj_module,
    t_out,
    traj_args,
    index_of_p=3,
    index_of_a=2,
    index_of_e=4,
    index_of_x=5,
    traj_kwargs={},
    xtol=2e-12,
    rtol=8.881784197001252e-16,
    bounds=None,
)

print(
    "p0 = {} will create a waveform that is {} years long, given the other input parameters.".format(
        p_new, t_out
    )
)

TypeError: not enough arguments: expected 4, got 3

In [13]:
# Jonathan's code

import numpy as np
import time
from tqdm import tqdm
from numpy.random import seed, uniform, randint

from few.trajectory.inspiral import EMRIInspiral
from few.trajectory.ode.flux import KerrEccEqFlux
from few.utils.utility import get_p_at_t

# 9700484.894655073, 25.806654874357793, 0.7735415084736439, 0.6834440767965327 stuck parameters
# 352224.8117850187, 63.9495298217371, 0.424046325083201, 0.7687664552808469

def gen_parameters(NEVAL, duration, seed_in=314159):

    traj_module = EMRIInspiral(func=KerrEccEqFlux)

    M_range = [1E5, 1E7]
    mu_range = [1,1E2]
    a_range = [0.0, 0.999]
    e_range = [0.0, 0.9]

    x0 = 1.0  # will be ignored in Schwarzschild waveform
    qK = np.pi / 3  # polar spin angle
    phiK = np.pi / 3  # azimuthal viewing angle
    qS = np.pi / 3  # polar sky angle
    phiS = np.pi / 3  # azimuthal viewing angle
    distance = 1.0  # distance
    # initial phases
    Phi_phi0 = np.pi / 3
    Phi_theta0 = 0.0
    Phi_r0 = np.pi / 3

    _base_params = [
        1E5, # M
        10,  # mu
        0.0, # a
        0.0, # p0
        0.0, # e0
        x0,
        distance,
        qS,
        phiS,
        qK,
        phiK,
        Phi_phi0,
        Phi_theta0,
        Phi_r0,
    ]

    seed(seed_in)
    M_seed, mu_seed, a_seed, e_seed = randint(1E3, 1E5, size=4)

    seed(M_seed)
    M_list = uniform(low=M_range[0], high=M_range[1], size=NEVAL)
    seed(mu_seed)
    mu_list = uniform(low=mu_range[0], high=mu_range[1], size=NEVAL)
    seed(a_seed)
    a_list = uniform(low=a_range[0], high=a_range[1], size=NEVAL)
    seed(e_seed)
    e_list = uniform(low=e_range[0], high=e_range[1], size=NEVAL)

    output_params_list = []
    failed_params_list = []

    for i, (M, mu, a, e) in enumerate(zip(
        M_list,
        mu_list,
        a_list,
        e_list,
    )):

        try:
            print(f"{i+1}:\t{M}, {mu}, {a}, {e}")
            updated_params = _base_params.copy()

            updated_params[0] = M
            updated_params[1] = mu
            updated_params[2] = a
            updated_params[4] = e
            updated_params[3] = get_p_at_t(
                traj_module,
                duration * 0.99,
                [updated_params[0], updated_params[1], updated_params[2], updated_params[4], 1.0],
                index_of_p=3,
                index_of_a=2,
                index_of_e=4,
                index_of_x=5,
                traj_kwargs={},
                xtol=2e-6,
                rtol=8.881784197001252e-6,
            )

            output_params_list.append(
                updated_params.copy()
            )
        except ValueError:
            failed_params_list.append([M, mu, a, e, duration])

    return output_params_list, failed_params_list

In [14]:
gen_parameters(50, 1., seed_in=314159)

1:	7528789.672533787, 18.599958825030917, 0.3752060897745056, 0.83726948312581


TypeError: not enough arguments: expected 4, got 3